In [1]:
cd ../..

/home/d/PycharmProjects/protein_properties


In [2]:
from copy import deepcopy
import json
from pathlib import Path
import sys
import pandas as pd
from typing import Optional
from tempfile import gettempdir
import numpy as np
from tqdm import tqdm
from biotite.structure.io.pdbx import PDBxFile, get_structure, get_sequence
import biotite.structure as biostruc
import biotite.database.rcsb as rcsb
from biotite.sequence import ProteinSequence, AlphabetError
from src.data.fasta import Fasta
from os import path
from src.utils import align_sequences_nw
import argparse
import multiprocessing as mp
import os
import tempfile

In [3]:
with open("data/substitution_dict.json", "r") as f:
        aa_dict = json.load(f)

codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']
# create matrix with diagonal 1
one_hot = np.identity(len(codes))
# create dataframe with one-hot encoding for each amino acid
one_hot = pd.DataFrame(one_hot, index=codes, columns=codes)
one_hot["AA"] = one_hot.index
one_hot_ss = {"a": [1, 0, 0], "b": [0, 1, 0], "c": [0, 0, 1]}

In [4]:
def create_dataset_ala_pandey(protein: str, 
                                 pdb_path: str): 
                                 
    cif_header: str = protein.split('-')[0]
    try:
        pdbx = PDBxFile.read(os.path.join(pdb_path, f'{cif_header}.cif'))
    except FileNotFoundError:
        print(f'Could not find PDBx file for {protein}\nFetching from RCSB...')
        file_path = rcsb.fetch(cif_header, "cif")
        pdbx = PDBxFile.read(file_path)
    struct = get_structure(pdbx, model=1, extra_fields=["b_factor"])
    """seq = get_sequence(pdbx, model=1)
    # Thank you biotite for this wonderful class, NOT!
    try:
        seq = ProteinSequence(list(("".join(protein_seq)).replace('U', 'C').replace("O", "K")))
    except AlphabetError:
        print(f"Protein {protein} contains non-canonical amino acids, skipping...")
        print(f"Protein sequence: {protein_seq}")
        sys.exit(1)

    seq_length = len(seq)"""
    chain_id = protein.split('-')[1]
    chain_starts = biostruc.get_chain_starts(struct).tolist()
    chain_ids = biostruc.get_chains(struct).tolist()
    if biostruc.get_chain_count(struct) == 1 or chain_starts[chain_ids.index(chain_id)] == chain_starts[-1]:
        struct = struct[chain_starts[chain_ids.index(chain_id)]:]
    else:
        struct = struct[chain_starts[chain_ids.index(chain_id)]:chain_starts[chain_ids.index(chain_id) + 1]]
    
    struct = struct[biostruc.filter_amino_acids(struct)]
    try:
        atom_sasa_scores = biostruc.sasa(struct, vdw_radii="Single", point_number=500)
    except ValueError:
        print(f'Skipping protein {protein}...\n')
        return protein, None, None

    bfactor = biostruc.apply_residue_wise(struct, struct.get_annotation("b_factor"), np.nanmean)
    bfactor = (bfactor - np.nanmean(bfactor)) / np.nanstd(bfactor)
    
    struct_seq = []
    for aa in biostruc.get_residues(struct)[1]:
        try:
            struct_seq.append(ProteinSequence.convert_letter_3to1(aa))
        except KeyError:
            try:
                struct_seq.append(aa_dict[aa])
            except KeyError:
                struct_seq.append('X')
    struct_ss = biostruc.annotate_sse(struct)
    ca_list = np.array([atom.coord for atom in struct if atom.atom_name == "CA"])
    try:
        assert len(ca_list) == len(struct_seq) == len(struct_ss), f"Length of PDB sequence ({len(seq)}) and CA atoms ({len(ca_list)}) and len SS ({len(struct_ss)}) do not match. Protein: {protein}"
        # TODO f this man, assert is triggered for 1 protein, need to investigate -> 115 protein affected by this -> not worth the time< 
        # I hate my life
    except AssertionError as e:
        print(e)
        print(f"Skipping protein {protein}...")
        return protein, None

    ca_coord_norm = (ca_list - np.nanmean(ca_list, axis=0)) / np.nanstd(ca_list, axis=0)
    struct_seq = [x if x in codes else "-" for x in struct_seq]
    one_hot_seq = np.array(one_hot.merge(pd.DataFrame(data={"AA": struct_seq}), how="right", on="AA").drop("AA", axis=1))
    struct_ss = np.array([one_hot_ss[x] for x in struct_ss])

    # !FIX: remove empty residues and adjust protein length accordingly  
    #!----------------------------------------------------------------
    # mask the residues that are not in the PDB files, due to disorder

    assert one_hot_seq.shape[0] == len(struct_ss) == len(ca_coord_norm), f"Length of one-hot sequence ({one_hot_seq.shape[0]}), secondary structure ({len(struct_ss)}) and CA coordinates ({len(ca_coord_norm)}) do not match"
    final_features = np.stack(np.concatenate([one_hot_seq, struct_ss, ca_coord_norm], axis=1))

    start_end_pp = np.zeros(final_features.shape[0])
    start_end_pp[0], start_end_pp[-1] = 1, 1
    final_features = np.concatenate([final_features, start_end_pp[:, np.newaxis]], axis=1)    
    prot_length = final_features.shape[0]
    final_features = np.pad(final_features, ((0, 500 - final_features.shape[0]), (0, 0)), mode='constant', constant_values=0)
    final_features = np.pad(final_features, ((0, 0), (0, 1)), mode='constant', constant_values=prot_length)

    #!Assert no empty residues
    
    assert np.where(~final_features.any(axis=1))[0].shape[0] == 0
    assert bfactor.shape[0] == len(struct_seq) == len(ca_coord_norm), f"Length of SASA ({bfactor.shape[0]}), secondary structure ({len(struct_ss)}) and CA coordinates ({len(ca_coord_norm)}) do not match"
    return protein, final_features, bfactor


In [5]:
map_missing_res = Fasta(path="data/substitution_dict.json")

global codes
codes = ['A', 'V', 'F', 'I', 'L','D','E','K','S','T','Y','C','N','Q', 'P','M', 'R', 'H', 'W', 'G', '-']
# create matrix with diagonal 1
one_hot_ss = {"a": [1, 0, 0], "b": [0, 1, 0], "c": [0, 0, 1]}
global one_hot
one_hot = np.identity(len(codes))
# create dataframe with one-hot encoding for each amino acid
one_hot = pd.DataFrame(one_hot, index=codes, columns=codes)
one_hot["AA"] = one_hot.index

fasta_path = "data/pandey_bfactor/seq_to_test_for.fasta"
# tmp path

temp_dir = tempfile.TemporaryDirectory()
print(temp_dir.name)
# use temp_dir, and when done:

pdb_path = temp_dir.name

global aa_dict
with open("data/substitution_dict.json", "r") as f:
    aa_dict = json.load(f)

all_ids = []
all_pandey_ids = []
all_seqs = []
with open(fasta_path, "r") as f:
    for line in f:
        if line.startswith(">"):
            pid = line[1:7].strip()
            pandey_id = line[7:].strip()
        else:
            all_ids.append(pid)
            all_pandey_ids.append(pandey_id)
            all_seqs.append(line.strip())

prots = []
final_features = []
bfactors = []
# all_ids = [x for xs in all_ids for x in xs]
for pid in all_ids:
    prot, final_feature, bf= create_dataset_ala_pandey(pid, pdb_path)
    if final_feature is not None:
        prots.append(prot)
        final_features.append(final_feature)
        bfactors.append(bf)
    else:
        print(f"Skipping {pid}")
        continue
    print(f"Done {pid}")
temp_dir.cleanup()

/tmp/tmprwo9ifpr
Could not find PDBx file for 4o75-A
Fetching from RCSB...
Done 4o75-A
Could not find PDBx file for 1lui-A
Fetching from RCSB...


/tmp/ipykernel_183136/1686213900.py:38: RuntimeWarning: invalid value encountered in divide
  bfactor = (bfactor - np.nanmean(bfactor)) / np.nanstd(bfactor)


Done 1lui-A
Could not find PDBx file for 1jek-A
Fetching from RCSB...
Done 1jek-A
Could not find PDBx file for 1jek-B
Fetching from RCSB...
Done 1jek-B
Could not find PDBx file for 101m-A
Fetching from RCSB...
Done 101m-A
Could not find PDBx file for 133l-A
Fetching from RCSB...
Done 133l-A


In [7]:
all_x = np.load("data/pandey_bfactor/x_61046.npy")
all_y = np.load("data/pandey_bfactor/y_61046.npy")

In [8]:
all_x[int(all_pandey_ids[0])][1]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        2.13764401e+00,  7.00131947e-01, -1.62279185e-02,  0.00000000e+00,
        1.27000000e+02])

In [9]:
final_features[0][1]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
       -7.08792448e-01,  2.11673951e+00,  7.59417564e-02,  0.00000000e+00,
        1.27000000e+02])

In [10]:
bfactors[0]

array([ 1.38153265e+00,  1.21513673e+00,  2.65594785e-01, -2.53427350e-01,
        2.27035513e-02, -3.85778112e-02, -2.48685340e-01,  7.62335551e-01,
        2.94821982e-01,  3.77199211e-01,  2.94958771e-01,  1.50737827e-01,
        1.60009023e-02, -4.82300269e-01, -1.71354097e-01,  8.02966837e-02,
        4.19419514e-04,  1.46026218e+00,  1.34499486e+00,  1.21354086e+00,
        2.97902263e-01, -3.37324453e-01,  9.10148467e-01, -2.60722750e-01,
       -3.55517358e-01, -2.41891498e-01,  1.49995579e+00, -3.98605816e-01,
       -7.21427279e-01, -6.44050440e-01,  1.11101516e+00, -5.88286224e-01,
       -1.09472379e+00, -5.46702442e-01, -4.14777286e-01, -5.43383035e-01,
       -4.53929268e-01, -4.76368151e-01, -8.55662644e-01, -7.31276069e-01,
       -9.73848129e-01, -8.49163106e-01, -8.23603413e-01, -7.86437402e-01,
       -7.91463122e-01, -5.21533311e-01, -1.01713463e-02, -2.83192583e-01,
        6.65367068e-03,  2.85277167e-01,  1.20370723e-01,  6.43496520e-01,
       -2.13165860e-01, -

In [11]:
all_y[1].squeeze()[:128].shape

(128,)

In [13]:
all_pandey_ids

['1', '50', '223', '223', '33714', '51818']

In [40]:
all_x[51818]

array([[  0.        ,   0.        ,   0.        , ...,  -0.9604993 ,
          1.        , 130.        ],
       [  0.        ,   1.        ,   0.        , ...,  -1.2813436 ,
          0.        , 130.        ],
       [  0.        ,   0.        ,   1.        , ...,  -1.24054866,
          0.        , 130.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 130.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 130.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 130.        ]])

In [8]:
from scipy.stats import pearsonr
pearsonr(all_x[1][:127, 26], final_features[0][:127, 26])

PearsonRResult(statistic=0.9999999999999997, pvalue=0.0)

In [5]:
ids = np.load("data/pandey_bfactor/pdb_ids.npy")
ids[50]

'1lun'